<a href="https://colab.research.google.com/github/HaraldsU/VTP_LPD/blob/main/Val_teh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip install stanza
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [ ]:
!pip install line_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
%load_ext line_profiler

In [ ]:
!wget "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords" -O 'stopwords.txt'
!wget "https://raw.githubusercontent.com/HaraldsU/VTP_LPD/main/input.csv"

--2024-06-17 13:38:24--  https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622 [text/plain]
Saving to: ‘stopwords.txt’

stopwords.txt       100%[===================>]     622  --.-KB/s    in 0s      

2024-06-17 13:38:25 (36.0 MB/s) - ‘stopwords.txt’ saved [622/622]

--2024-06-17 13:38:25--  https://raw.githubusercontent.com/HaraldsU/VTP_LPD/main/input.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
import stanza
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)
from stanza.server import CoreNLPClient

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

## Functions

In [ ]:
def format_text(output):
  # print_help(output, 'Op')
  regexp = re.compile(r'[MSK\d*]')
  regexp2 = re.compile(r"'[A-Za-z]")
  final_output = []

  for word in range(len(output)):
    prev = output[word - 1]
    pprev = output[word - 2]
    cur = output[word]

    if not regexp.search(cur):
      if word == 0:
        cur = cur.capitalize()
      elif word > 1 and prev in ['.', '!', '?']:
        cur = cur.capitalize()

    if (cur in [".", ",", '!', '?', ':', ';'] and final_output) or regexp2.search(cur):
      final_output[-1] += cur
    elif cur == '-':
      final_output[-1] += cur
      final_output[-1] += output[word + 1]
      word += 1
    elif word == len(output) - 1 and cur not in ['.', '?', '!', '\'']:
      final_output[-1] += '.'
    else:
        final_output.append(cur)

  return ' '.join(final_output)

In [ ]:
def spans_to_ranges(spans):
  msk_gr = []
  for span in spans:
    span1_start = span['span1']['start']
    span1_end = span['span1']['end']
    span2_start = span['span2']['start']
    span2_end = span['span2']['end']
    in_grp = False
    c = 0
    for gr in msk_gr:
      for rng in gr:
        if span1_start<=rng[1] and span1_end >= rng[0]:
          c+=1
        if span2_start<=rng[1] and span2_end >= rng[0]:
          c+=2
        if  c == 3 : break
      if    c == 0 :
        continue
      elif  c == 1 :
        gr.append([span2_start,span2_end])
        break
      elif  c == 2 :
        gr.append([span1_start,span1_end])
        break
      elif  c == 3 :
        break
    if c == 0: msk_gr.append([[span1_start,span1_end],[span2_start,span2_end]])
  return msk_gr

In [ ]:
def get_similar_spans(spans, cos_threshold, model):
  embedings =  model.encode([s["text"] for s in spans])
  similar_spans = []
  for i in range(len(spans)-1):
    emb = embedings[i]
    for j in range(i+1,len(spans)):
      if spans[j]['start'] <= spans[i]['end']:
        continue
      emb_chk = embedings[j]
      cos_score = cosine_similarity([emb], [emb_chk])
      if(cos_score>=cos_threshold):
        score = 1*cos_score + 0.15*(1-1/(spans[i]['end']+1-spans[i]['start']+spans[j]['end']+1-spans[j]['start'])) # cos_score +  length of text

        similar_spans.append({
            'score': score, 'cos:':cos_score
            ,'span1':spans[i]
            ,'span2':spans[j]})
  similar_spans = sorted(similar_spans, key=lambda d: d['score'], reverse=True)
  return similar_spans

In [ ]:
def get_spans(words, stopwords):
  spans=[]
  regexp = re.compile(r'[MSK\d*]')
  range(len(words))
  for i in range(len(words)):
    for j in range(i+1, len(words)+1):
      c = False
      for w in words[i:j]:
        if w in stopwords:
          c = True
          break
      if c: continue

      s = ' '.join(words[i:j])
      # s=s.replace(' ,','')
      if(regexp.search(s) or s ==','):
        continue
      spans.append({"text":s, "start":i, "end": j-1})
  return spans

In [ ]:
def coref_mask_and_tokenize(text, core_nlp_client):
  msk_id=0
  document = core_nlp_client.annotate(text)
  corefs = document.corefChain
  resolved = []
  chain_mask = {}
  for chain in corefs:
    if len(chain.mention) == 1:
      continue
    msk_id+=1
    chain_mask[chain.chainID]='[MSK'+str(msk_id)+']'

  words = []
  words_lemmatized = []
  for sentence in document.sentence:
    for token in sentence.token:
        corefClustId = token.corefClusterID
        chain = None
        for c in corefs:
          if c.chainID == corefClustId:
            chain = c
            break

        if chain is None or len(chain.mention) == 1:
          word = token.word.lower()
          word_l = token.lemma
        else:
          word = chain_mask.get(chain.chainID)
          word_l = word
        words.append(word)
        words_lemmatized.append(word_l)
  return words, words_lemmatized, msk_id


In [ ]:
# def get_stopwords():
#   text_file = open("stopwords.txt", "r")
#   stopwords = text_file.read().splitlines()
#   stopwords.extend(['.',',','!','?','therefore','if', 'given', 'give', ':', '\'', '\"', ' '])
#   text_file.close()
#   return stopwords

def get_stopwords():
    # Open the file and read the stopwords
    with open("stopwords.txt", "r") as text_file:
        stopwords = text_file.read().splitlines()

    # List of additional stopwords to add
    additional_stopwords = ['.', ',', '!', '?', 'therefore', 'if', 'given', 'give',
                            ':', '\'', '\"', '”']

    # Extend the stopwords list if they are not already present
    for word in additional_stopwords:
        if word not in stopwords:
            stopwords.append(word)

    # Write the updated list back to the file
    with open("stopwords.txt", "w") as text_file:
        for word in stopwords:
            text_file.write(word + "\n")

    return stopwords

In [ ]:
def remove_quotes(text):
    result = []
    for word in text:
        new_word = word.replace('"', '').replace('”', '').replace('“', '')
        result.append(new_word)
    return result

In [ ]:
def print_help(obj, name='Obj'):
  print(name, '=', end=" ")
  for item in obj:
    print(item, end =" "),
  print()

In [ ]:
def mask_word_ranges(words, mask_ranges, msk_index):
  d = dict(enumerate(map(str, words)))
  for msk in mask_ranges:
    msk_index+=1
    msk_txt = '[MSK'+str(msk_index)+']'
    for r in msk:
      d[r[0]]= msk_txt
      for i in range(r[0]+1,r[1]+1):
        d.pop(i)
  arr = [v for k, v in d.items()]
  return arr

In [ ]:
def mask_text(text_arr, core_nlp_client, out_file_name, write_mode, model):
  stopwords = get_stopwords()

  text_arr = remove_quotes(text_arr)

  result = []
  write_queue=[]
  total = len(text_arr)
  i = 0

  for text in text_arr:
    i+=1
    words, words_lemmatized, mask_count = coref_mask_and_tokenize(text, core_nlp_client)
    spans = get_spans(words_lemmatized, stopwords)
    print(f'\r{round((i / total) * 100, 2)}% / 100% (', i , ')', end='', flush=True)

    if not spans:
      result=text
      write_queue.append(result+'\n')
      print('\nNo valid spans')
      continue

    similar_spans = get_similar_spans(spans, .91, model)
    mask_ranges = spans_to_ranges(similar_spans)
    maksed_words = mask_word_ranges(words, mask_ranges, mask_count)
    result = format_text(maksed_words)
    write_queue.append(result + '\n')

  write_queue_df = pd.DataFrame(write_queue)
  write_queue_df.to_csv(out_file_name, index=False, quoting=1)
  print('\nFinished!')


## Server

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'coref'],
    memory='4G',
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)
client.start()

## Execute

- Pieliku klat profileri (visu laiku patere coref_mask_and_tokenize() un get_similar_spans());
- Uzliku skaistaku progresa printu;
- Pieliku get_stopwords klat paris stopwordus;
- Ieliku savu format_text, jo bija problemas ar lielajiem burtiem, punktiem, pēdiņām utt.
- Pieliku metodi remove_quotes.
- Pamainiju mask_text faila izveidi.
- output.csv fails ar 1108 rindinam:
https://github.com/HaraldsU/VTP_LPD/blob/main/output.csv


In [ ]:
# text = "Jack is a good athlete. Jack comes from Canada. Therefore, all Canadians are good athletes. Good athletes always win." # [MSK1] is a [MSK2]. [MSK1] comes from [MSK3]. Therefore, all [MSK3] are [MSK2].
text = "If Joe eats greasy food, he will feel sick. Given now that Joe feels sick, therefore, Joe must have had greasy food." #
# text = ["McDonald's Hamburgers: over 99 billion served."]
# text = ['men don’t cry']
# text = ['David is so wrong about Luna\'s work ethic. David is just an egotistical jerk with a God complex, what does he know?']

model = SentenceTransformer('bert-base-nli-mean-tokens')

df = pd.read_csv('input.csv')
# selected_rows = df.iloc[0:99] # First 99 rows
rows = [text]#df['source_article']
print('Row count = ', len(rows))

%lprun -f mask_text mask_text(rows, client, "output.csv", 'w', model)

In [ ]:
client.stop()